In [ ]:
import numpy as np
import pandas as pd
import mplfinance as mpf

df = pd.read_csv('auto-indig.csv')  # assumes the file is in the current directory

Query = """
Close above EMA21
Rsi_14 above_lvl 30
"""

# Split into lines and then split each line into words
arr = np.array([line.split() for line in Query.splitlines() if line.strip() != ""])

actlist = arr[:, arr.shape[1] // 2].tolist()

# actlist
arr

In [ ]:
def above(x, y, new_col=None):
    """
    Compare two columns from global df and add a new column with 1/0.

    Parameters:
    x       : str : name of first column
    y       : str : name of second column
    new_col : str : name of the new column to store result (optional)

    Returns:
    pd.DataFrame : df with the new column added
    """
    if new_col is None:
        new_col = f"{x}_above_{y}"  # default column name

    df[new_col] = np.where(df[x] > df[y], 1, 0)
    return df

# Example usage
result_df = above("Close", "EMA_21")


In [ ]:
def below(x, y, new_col=None):
    """
    Compare two columns from global df and add a new column with 1/0 if x < y.

    Parameters:
    x       : str : name of first column
    y       : str : name of second column
    new_col : str : name of the new column to store result (optional)

    Returns:
    pd.DataFrame : df with the new column added
    """
    if new_col is None:
        new_col = f"{x}_below_{y}"  # default column name

    df[new_col] = np.where(df[x] < df[y], 1, 0)
    return df


In [ ]:
def crossed_up(x, y, new_col=None):
    """
    Detect where column x crosses above column y (e.g., Close crossing above EMA21)
    by checking sign changes in the difference.

    Parameters:
    x       : str : name of first column (e.g., 'Close')
    y       : str : name of second column (e.g., 'EMA21')
    new_col : str : name of the new column to store result (optional)

    Returns:
    pd.DataFrame : df with the new column added (1 = cross up, 0 = no cross)
    """
    if new_col is None:
        new_col = f"{x}_cross_up_{y}"

    diff = df[x] - df[y]
    df[new_col] = ((diff > 0) & (diff.shift(1) <= 0)).astype(int)
    return df


In [ ]:
def crossed_dn(x, y, new_col=None):
    """
    Detect where column x crosses below column y (e.g., Close crossing below EMA21)
    by checking sign changes in the difference.

    Parameters:
    x       : str : name of first column (e.g., 'Close')
    y       : str : name of second column (e.g., 'EMA21')
    new_col : str : name of the new column to store result (optional)

    Returns:
    pd.DataFrame : df with the new column added (1 = cross down, 0 = no cross)
    """
    if new_col is None:
        new_col = f"{x}_cross_down_{y}"

    diff = df[x] - df[y]
    df[new_col] = ((diff < 0) & (diff.shift(1) >= 0)).astype(int)
    return df


In [ ]:
def rejected(df, x='Close', y='EMA_21', new_col='reject'):
    """
    Automatically detect rejects:
    - Upward reject: x crosses above y, next candle closes below y
    - Downward reject: x crosses below y, next candle closes above y

    Marks 1 for upward reject, -1 for downward reject, 0 otherwise.

    Parameters:
    df      : pd.DataFrame : DataFrame containing columns x and y
    x       : str          : Column that might get rejected
    y       : str          : Column to be rejected
    new_col : str          : Name of the new column to store signals

    Returns:
    pd.DataFrame : Copy of df with new_col added
    """
    df = df.copy()

    # Upward rejection: x above y, next candle below y
    up_reject = (df[x] > df[y]) & (df[x].shift(-1) < df[y].shift(-1))

    # Downward rejection: x below y, next candle above y
    down_reject = (df[x] < df[y]) & (df[x].shift(-1) > df[y].shift(-1))

    # Combine: 1 = up reject, -1 = down reject, 0 = no reject
    df[new_col] = 0
    df.loc[up_reject, new_col] = 1
    df.loc[down_reject, new_col] = -1

    return df


In [ ]:
def cabr(action, clm1, clm2):
    match action:
        case "above":
            return above(clm1, clm2)
        case "below":
            return below(clm1, clm2)
        case "crossed_dn":
            return crossed_up(clm1, clm2)
        case "crossed_up":
            return crossed_dn(clm1, clm2)
        case "rejected":
            return rejected(clm1, clm2)
        case _:
            return "err"

In [ ]:

# cabr("above", "Close", "EMA_21")
# cabr("crossed_dn", "SMA_10", "EMA_21")

df.T.iloc[:,100:]

In [ ]:
pltdf = df.iloc[50:110].copy()

pltdf['DateTime'] = pd.to_datetime(pltdf['DateTime'])
pltdf.set_index('DateTime', inplace=True)

pltdf = rejected(pltdf, x='SMA_10', y='EMA_21', new_col='EMA10_reject_auto')


# Check results
pltdf[['Close', 'EMA_21', 'EMA10_reject_auto']].tail(10)


In [ ]:
import numpy as np
import mplfinance as mpf

# Create marker series for plotting
up_rejects = pltdf['EMA10_reject_auto'].replace(-1, np.nan).replace(0, np.nan)  # only 1s remain
down_rejects = pltdf['EMA10_reject_auto'].replace(1, np.nan).replace(0, np.nan)  # only -1s remain

# Use price values for marker placement
up_values = pltdf['Close'].where(up_rejects == 1)
down_values = pltdf['Close'].where(down_rejects == -1)

# Addplots for EMA, and reject markers
apds = [
    mpf.make_addplot(pltdf['EMA_21'], color='blue'),
    # mpf.make_addplot(pltdf['Close'], color='orange'),
    mpf.make_addplot(up_values, type='scatter', markersize=100, marker='^', color='green'),
    mpf.make_addplot(down_values, type='scatter', markersize=100, marker='v', color='red')
]

# Plot candlestick with rejected candles marked
mpf.plot(
    pltdf,
    type='candle',
    style='yahoo',
    title='EMA10 Rejected Candles',
    ylabel='Price',
    addplot=apds
)
